# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [24]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [32]:
import numpy as np
from datetime import datetime

def date_sorter():
    
    # Extract dates in MM/DD/YY and MM/DD/YYYY formats
    dates_mmddyy = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    dates_mmddyyyy = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    combined_dates_1 = pd.concat([dates_mmddyy, dates_mmddyyyy])
    combined_dates_1.reset_index(inplace=True)
    indices_dates_1 = combined_dates_1['level_0']
    
    # Extract dates in formats like "Month DD, YYYY"
    dates_month_day_year = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    dates_month_day_year.reset_index(inplace=True)
    indices_dates_2 = dates_month_day_year['level_0']
    
    # Extract dates in formats like "DD Month YYYY"
    dates_day_month_year = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    dates_day_month_year.reset_index(inplace=True)
    indices_dates_3 = dates_day_month_year['level_0']
    
    # Extract dates in formats like MM/YYYY
    dates_month_year = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    dates_month_year.reset_index(inplace=True)
    indices_dates_4 = dates_month_year['level_0']
    remaining_indices = []
    for i in indices_dates_4:
        if not (i in indices_dates_1.values):
            remaining_indices.append(i)
    remaining_indices = np.asarray(remaining_indices)
    dates_month_year = dates_month_year[dates_month_year['level_0'].isin(remaining_indices)]

    # Extract years only
    years_only_1 = df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    years_only_2 = df.str.extractall(r'^(\d{4})[^0-9]')
    combined_years = pd.concat([years_only_1, years_only_2])
    combined_years.reset_index(inplace=True)
    indices_years = combined_years['level_0']
    remaining_indices_years = []
    for i in indices_years:
        if not ((i in indices_dates_2.values) | (i in indices_dates_3.values) | (i in indices_dates_4.values)):
            remaining_indices_years.append(i)
    remaining_indices_years = np.asarray(remaining_indices_years)
    combined_years = combined_years[combined_years['level_0'].isin(remaining_indices_years)]
    
    # Combine all indices
    all_indices = combined_dates_1.level_0.values.tolist() + \
                  dates_month_day_year.level_0.values.tolist() + \
                  dates_day_month_year.level_0.values.tolist() + \
                  dates_month_year.level_0.values.tolist() + \
                  combined_years.level_0.values.tolist()
    all_indices = np.asarray(all_indices)
    
    # Rename and process extracted columns
    combined_dates_1.columns = ['level_0', 'match', 'month', 'day', 'year']
    combined_dates_1['year'] = combined_dates_1['year'].apply(str)
    combined_dates_1['year'] = combined_dates_1['year'].apply(lambda x: '19' + x if len(x) <= 2 else x)
   
    dates_month_day_year[1] = dates_month_day_year[1].apply(lambda x: x.replace(',', ''))
    dates_month_day_year['day'] = dates_month_day_year[1].apply(lambda x: x.split(' ')[0])
    dates_month_day_year['year'] = dates_month_day_year[1].apply(lambda x: x.split(' ')[1])
    dates_month_day_year.columns = ['level_0', 'match', 'month', 'day-year', 'day', 'year']
    dates_month_day_year.drop('day-year', axis=1, inplace=True) 
    
    dates_day_month_year.columns = ['level_0', 'match', 'day', 'month', 'year']
    dates_day_month_year['day'] = dates_day_month_year['day'].replace(np.nan, -99)
    dates_day_month_year['day'] = dates_day_month_year['day'].apply(lambda x: 1 if int(x) == -99 else x)
    dates_day_month_year['month'] = dates_day_month_year['month'].apply(lambda x: x[:3])
    dates_day_month_year['month'] = pd.to_datetime(dates_day_month_year['month'], format='%b').dt.month
    
    dates_month_year.columns = ['level_0', 'match', 'month', 'year']
    dates_month_year['day'] = 1
  
    combined_years.columns = ['level_0', 'match', 'year']
    combined_years['day'] = 1
    combined_years['month'] = 1
   
    # Combine all extracted and normalized dates
    final_data = pd.concat([combined_dates_1, dates_month_day_year, dates_day_month_year, dates_month_year, combined_years])
    final_data['date'] = pd.to_datetime(final_data['month'].apply(str) + '/' + final_data['day'].apply(str) + '/' + final_data['year'].apply(str))
    final_data = final_data.sort_values(by='level_0').set_index('level_0')

    # Create final answer
    sorted_dates = final_data['date']
    answer = pd.Series([i[0] for i in sorted(enumerate(sorted_dates), key=lambda x: x[1])], np.arange(500))
    return answer


In [33]:
print(date_sorter())

0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64
